In [57]:
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

# Assignment 1

# Loading Datasets

In [14]:
data1 = pd.read_csv("./ds-boot-2.csv", sep='\t')

In [15]:
data1.head()

,id,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p24,p25,p26,p27,p28,y1,y2,y3,y4,y5
0,11,6.48148,3.0,5.0,7.75000,0.0,7.16667,8.16667,9.66667,6.16667,...,1.66667,3.16667,0.0,0.00000,0.0,1.0,5.00,7.44,1.18,4.38
1,12,5.74074,4.0,8.0,7.33333,8.0,8.83333,9.75000,9.66667,9.00000,...,2.50000,5.50000,5.0,8.66667,8.0,4.5,4.25,8.93,2.00,6.03
2,25,7.59259,7.0,8.0,7.66667,8.0,9.66667,9.50000,6.16667,9.66667,...,3.50000,3.50000,9.0,6.50000,7.0,7.5,11.00,8.97,2.00,9.12
3,31,5.96297,4.0,8.0,9.33333,10.0,9.33333,7.00000,8.50000,9.66667,...,0.00000,0.00000,0.0,0.00000,0.0,4.0,6.25,8.93,1.82,6.41
4,48,5.44444,1.0,3.5,6.41667,9.0,8.50000,7.08333,6.33333,9.00000,...,0.00000,0.00000,0.0,0.00000,0.0,0.0,3.00,8.08,1.36,3.67


# Estimating Mean, Median, Error and confidence Interval

In [16]:
def bootstrap(X, B=1000, s=np.mean):
    sample = np.random.choice(X, size=(B, len(X)))
    estimations = s(sample, axis=1)
    est = estimations.mean()
    error = (((estimations - est) ** 2).sum() / (B - 1)) ** 0.5
    conf_int = np.percentile(estimations, [2.5, 97.5])
    return est, error, conf_int

In [19]:
print(bootstrap(data1['y1'], s=np.mean))

(4.0254124999999998, 0.43989813760143975, array([ 3.1996875,  4.8503125]))


In [20]:
print(bootstrap(data1['y1'], s=np.median))

(4.0054999999999996, 0.54407330918339103, array([ 3.,  6.]))


In [25]:
target = ['y1', 'y2', 'y3', 'y4', 'y5']

In [29]:
for label in target:
    print(label, 'mean:', bootstrap(data1[label], s=np.mean))
    print(label, 'meadian:', bootstrap(data1[label], s=np.median))

('y1', 'mean:', (4.0225625000000003, 0.44275302945190437, array([ 3.1371875,  4.9003125])))
('y1', 'meadian:', (4.0182500000000001, 0.53783706287890409, array([ 3.,  6.])))
('y2', 'mean:', (5.5227937499999999, 0.53026086267130679, array([ 4.4809375,  6.5753125])))
('y2', 'meadian:', (6.1651249999999997, 0.66438657338472307, array([ 4.625,  7.25 ])))
('y3', 'mean:', (7.3128430000000009, 0.45754116825335689, array([ 6.35858125,  8.11305625])))
('y3', 'meadian:', (8.3044249999999984, 0.27978818023783802, array([ 7.96,  8.92])))
('y4', 'mean:', (1.5286759999999999, 0.10993528398865246, array([ 1.29313125,  1.72755   ])))
('y4', 'meadian:', (1.8497199999999998, 0.13321399163589226, array([ 1.64,  2.  ])))
('y5', 'mean:', (5.6435752499999996, 0.45151077922757971, array([ 4.7133125 ,  6.55406875])))
('y5', 'meadian:', (6.1481299999999992, 0.44503872029584418, array([ 5.09   ,  7.04625])))


# Linear Reagression of Initial Sample

In [31]:
features = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6']

In [34]:
for label in target:
    model = LinearRegression()
    model.fit(data1[features], data1[label])
    print(label, model.coef_)

('y1', array([ 0.19590832,  0.29677494, -0.18401943,  0.02210887,  0.13189801,
        0.28986106]))
('y2', array([ 0.12744323,  0.18071353,  0.03383592,  0.10825791, -0.05377676,
        0.57520423]))
('y3', array([ 0.15867643,  0.01838332,  0.12772208,  0.29122447,  0.17922699,
        0.23654851]))
('y4', array([-0.00486422,  0.00423981,  0.11801823,  0.02819031,  0.06902863,
        0.00098964]))
('y5', array([ 0.152759  ,  0.16444265,  0.01372641,  0.1348942 ,  0.10091446,
        0.34552193]))


In [41]:
''' returns estimations of coefficients (with intercept) and 95% confidence intervals'''

' returns estimations of coefficients (with intercept) and 95% confidence intervals'

In [46]:
def coeffs_bootstrap(X, Y, B=1000, model_class=LinearRegression):
    
    n = X.shape[0]
    coeffs = []
    for b in range(B):
        model = model_class()
        idx = np.random.randint(n, size=n)
        model.fit(X.as_matrix()[idx, :], Y.as_matrix()[idx])
        coeffs.append(np.append(model.coef_, model.intercept_))
    coeffs = np.array(coeffs)
    conf_int = np.percentile(coeffs, [2.5, 97.5], axis=0)
    return coeffs.mean(axis=0), conf_int


for label in targets:
    coeffs, intervals = coeffs_bootstrap(data1[features], data1[label])
    print(label,
          'coef', coeffs[:-1], 'inpt', coeffs[-1],
          'from', intervals[0][:-1], '    ', intervals[0][-1],
          'to  ', intervals[1][:-1], '    ', intervals[1][-1])

('y1', 'coef', array([ 0.22463472,  0.28980205, -0.10382083,  0.08125146,  0.11644295,
        0.15215769]), 'inpt', -0.41910306506748296, 'from', array([ -1.40856474e-01,  -2.29086371e-04,  -5.63793913e-01,
        -4.16708187e-01,  -4.28611544e-01,  -6.51422751e-01]), '    ', -1.5770405968985788, 'to  ', array([ 0.64207573,  0.56339271,  0.45155496,  0.63688043,  0.50980624,
        0.63016197]), '    ', 0.19862459178211342)
('y2', 'coef', array([ 0.12663644,  0.1667847 ,  0.12882126,  0.15719443, -0.05688789,
        0.4505047 ]), 'inpt', -0.57658408788651783, 'from', array([-0.33377816, -0.17864577, -0.47830489, -0.30100218, -0.5197511 ,
       -0.40707145]), '    ', -1.8721329917677563, 'to  ', array([ 0.62772802,  0.51895294,  0.9030343 ,  0.61496594,  0.32975561,
        0.89073681]), '    ', 0.044147174173777975)
('y3', 'coef', array([ 0.14284834,  0.02293654,  0.10997328,  0.29424803,  0.19422908,
        0.25047633]), 'inpt', 0.59394802278581382, 'from', array([-0.02319219, -

# Assignment 2

# Loading Datasets

In [49]:
data2 = pd.read_csv("./ds-boot-2.csv", sep='\t')

In [52]:
data2

,id,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p24,p25,p26,p27,p28,y1,y2,y3,y4,y5
0,11,6.48148,3.0,5.00,7.75000,0.00000,7.16667,8.16667,9.66667,6.16667,...,1.66667,3.16667,0.00000,0.00000,0.00000,1.0,5.00,7.44,1.18,4.38
1,12,5.74074,4.0,8.00,7.33333,8.00000,8.83333,9.75000,9.66667,9.00000,...,2.50000,5.50000,5.00000,8.66667,8.00000,4.5,4.25,8.93,2.00,6.03
2,25,7.59259,7.0,8.00,7.66667,8.00000,9.66667,9.50000,6.16667,9.66667,...,3.50000,3.50000,9.00000,6.50000,7.00000,7.5,11.00,8.97,2.00,9.12
3,31,5.96297,4.0,8.00,9.33333,10.00000,9.33333,7.00000,8.50000,9.66667,...,0.00000,0.00000,0.00000,0.00000,0.00000,4.0,6.25,8.93,1.82,6.41
4,48,5.44444,1.0,3.50,6.41667,9.00000,8.50000,7.08333,6.33333,9.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,3.00,8.08,1.36,3.67
5,101,9.62963,10.0,8.00,7.00000,6.66667,9.00000,8.41667,7.00000,0.00000,...,0.00000,0.00000,0.00000,8.00000,0.00000,4.0,6.25,8.04,1.27,6.00
6,151,0.00000,0.0,0.00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00,0.00,0.00,0.00
7,155,7.07408,4.0,6.00,5.75000,7.66667,6.33333,5.33333,3.33333,8.00000,...,9.00000,10.00000,8.00000,10.00000,10.00000,2.0,3.00,6.30,2.00,3.99
8,177,9.25926,0.0,7.50,9.75000,7.00000,9.50000,7.08333,3.66667,7.33333,...,0.00000,0.00000,0.00000,0.00000,0.00000,4.0,7.00,8.03,1.45,6.27
9,193,0.00000,0.0,0.00,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.0,0.00,0.00,0.00,0.00


In [53]:
data2.head()

,id,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p24,p25,p26,p27,p28,y1,y2,y3,y4,y5
0,11,6.48148,3.0,5.0,7.75000,0.0,7.16667,8.16667,9.66667,6.16667,...,1.66667,3.16667,0.0,0.00000,0.0,1.0,5.00,7.44,1.18,4.38
1,12,5.74074,4.0,8.0,7.33333,8.0,8.83333,9.75000,9.66667,9.00000,...,2.50000,5.50000,5.0,8.66667,8.0,4.5,4.25,8.93,2.00,6.03
2,25,7.59259,7.0,8.0,7.66667,8.0,9.66667,9.50000,6.16667,9.66667,...,3.50000,3.50000,9.0,6.50000,7.0,7.5,11.00,8.97,2.00,9.12
3,31,5.96297,4.0,8.0,9.33333,10.0,9.33333,7.00000,8.50000,9.66667,...,0.00000,0.00000,0.0,0.00000,0.0,4.0,6.25,8.93,1.82,6.41
4,48,5.44444,1.0,3.5,6.41667,9.0,8.50000,7.08333,6.33333,9.00000,...,0.00000,0.00000,0.0,0.00000,0.0,0.0,3.00,8.08,1.36,3.67


In [55]:
data2.describe()

,id,p1,p2,p3,p4,p5,p6,p7,p8,p9,...,p24,p25,p26,p27,p28,y1,y2,y3,y4,y5
count,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000,40.000000
mean,378.275000,6.264353,4.812500,6.106250,6.854167,6.537501,6.825000,6.768750,6.262501,6.516668,...,3.187500,3.722917,3.483333,3.916667,3.254167,4.025000,5.506250,7.306000,1.527500,5.634750
std,204.267545,3.168813,4.028246,3.210004,3.105559,3.114045,3.249140,3.144689,3.002584,3.585902,...,3.964980,4.055499,4.192620,4.339287,4.256550,2.857626,3.419161,2.956749,0.700848,2.811536
min,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,205.750000,5.458337,0.000000,4.375000,6.541665,6.000000,6.249998,6.374998,5.916667,5.375000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000,7.277500,1.427500,4.155000
50%,414.500000,6.666670,4.000000,7.500000,7.666670,7.000000,8.000000,7.833335,7.000000,8.000000,...,1.333335,3.333335,0.000000,1.708335,0.000000,4.000000,6.250000,8.215000,1.865000,6.225000
75%,563.250000,8.726850,9.125000,8.500000,8.687503,9.000000,9.000000,8.812500,8.125000,9.000000,...,6.041667,6.625000,7.250002,8.541667,7.625000,6.500000,8.312500,9.072500,2.000000,8.025000
max,638.000000,10.000000,10.000000,10.000000,9.750000,10.000000,10.000000,10.000000,9.666670,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,11.000000,9.720000,2.000000,9.490000


In [64]:
plt.plot(data2['cummBugs'])
plt.show()

KeyError: 'cummBugs'